In [1]:
from sympy import *
import numpy as np
import casadi as ca
import pickle


# This is a painfull work about qudrupedal robot dynamics investigation.
Kovalev V.V

## Task description
Two koordinate frames: world and body.
All quantities without $ B $ subscription are in world frame. With $B$ are in body frame.

State: $[p,\theta,\dot{p},\omega_B]$

Parameters: $[u_i,p^u_i]; i = 1,2,3,4$

$p \in \R^3$ - position of Center if mass (CoM); 
$\dot{p} \in \R^3$ - velocity of CoM; 
$\theta \in \R^3$ is rpy angels;
$\omega \in \R^3$ is the angular velocity;
$ I \in \R^{3 \times 3}$ is Inertia matrix;
$ u_i$ ground reaction force in position $p^u_i$ 

To come from body frame to world frame the rotation matrix is applyed: $R(pry) = R_zR_yR_x$

## Energies
Kinetic energy $T = (\dot{p}^T M\dot{p} +\omega^T I \omega)/2$

Potential energy $V = g^TMp$

$L = T - V; q = [p,\theta]$

after taking $\frac{d}{dt} \frac{\partial L}{\partial \dot{q}} - \frac{\partial L}{\partial q}$ considering $\frac{d}{dt} I\omega = I\dot{\omega} + \omega \times I\omega$

obtain 

$\ddot{p} = \sum{f}/m - g$

$I\dot{\omega} = \sum{r_i \times f_i} - \hat{\omega} I \omega$

where $ r_i = p_i^u - p $ are levers

Also it imoprtant to know the equality $\dot{R}R^T=\hat{\omega}$ where $\hat{\omega}$ is a skew matrix of $\omega$

$r(t) = R(t) r_B(0);  I = RI_BR^T$

$r(t) = R(t) r_B(0) => \dot{r}(t) = \dot{R(t)} r_B(0) => \dot{r}(t) = \dot{R(t)} R^T(t)r(t) =>\dot{r}(t) = \omega \times r(t)$

$\dot{R(t)} R^T(t) = \hat{\omega}$

$\dot{r}(t) = \hat \omega r(t) => R^T\dot{r}(t) = R^T\hat \omega r(t) =>\dot{r_B}(t) = R^T\hat \omega R(t) r_B(0) => \dot{r_B}(t) = \hat\omega_B r_B(0)$

$\hat\omega_B = R^T\hat \omega R$ and $\hat\omega = R\hat \omega_B R^T$

But according to sources $\omega = R \omega_B$ and $\omega_B = R^T \omega$

summing up:

$I\dot{\omega} = RI_BR^T \dot{R \omega_B} = RI_BR^T (\dot{R}\omega_B + R\dot{\omega_B}) = RI_BR^T (\dot{R} R_T \omega + R\dot{\omega_B})=RI_BR^T (\hat\omega \omega + R\dot{\omega_B}) = RI_BR^T R\dot{\omega_B} = RI_B\dot{\omega_B} $

$\hat{\omega} I \omega = \hat{\omega} I \omega = R\hat \omega_B R^T RI_BR^T R \omega_B =  R\hat \omega_B I_B\omega_B$

$I\dot{\omega} = \sum{r_i \times f_i} - \hat{\omega} I \omega$ could be expressed as follows:

$RI_B\dot{\omega_B} = \sum{r_i \times f_i} - R\hat \omega_B I_B\omega_B$

$\dot{\omega_B} = I_B^{-1}(R^T\sum{r_i \times f_i} - \hat \omega_B I_B\omega_B)$

Also it is important to know $\theta$ <- $ \omega_B$ transformation rule

$\hat\omega = \dot{R} R^T => \hat\omega[i,:] = \frac{\partial R}{\partial \theta_i} \dot\theta_i R^T => \hat\omega[i,:] = \frac{\partial R}{\partial \theta_i} R^T \dot\theta_i $

$\hat\omega = R\hat \omega_B R^T = \frac{\partial R}{\partial \theta_i} R^T \dot\theta_i => \hat \omega_B = R^T\frac{\partial R}{\partial \theta_i} \dot\theta_i$

$\dot\theta_i = [R^T\frac{\partial R}{\partial \theta_i}]^{-1}\hat \omega_B$

Let say there is $J \in \R^{3 \times 3}$ such that $J[:,i] = R^T\frac{\partial R}{\partial \theta_i}$

$\dot\theta = J^{-1} \hat\omega_B$

## Final equstions:

$\dot p = \dot p$

$\dot\theta = J^{-1} \hat\omega_B$

$\ddot{p} = \sum{f}/m - g$

$\dot{\omega_B} = I_B^{-1}(R^T\sum{r_i \times f_i} - \hat \omega_B I_B\omega_B)$

sources:

### quad dynamics : 

https://ieeexplore.ieee.org/abstract/document/8793669

https://arxiv.org/pdf/2209.14138.pdf

be aware that $\omega$ in another frame:
https://dspace.mit.edu/bitstream/handle/1721.1/138000/convex_mpc_2fix.pdf?sequence=2&isAllowed=y

### used linkes, books

https://www.brown.edu/Departments/Engineering/Courses/En4/Notes/Rigid_Bodies_1/Rigid_Bodies.pdf

https://physics.stackexchange.com/questions/672712/angular-velocity-via-extrinsic-euler-angles#mjx-eqn-B-08

https://physics.stackexchange.com/questions/73961/angular-velocity-expressed-via-euler-angles

https://physics.stackexchange.com/questions/410823/euler-lagrange-equations-for-free-rigid-body

https://physics.stackexchange.com/questions/672712/angular-velocity-via-extrinsic-euler-angles#mjx-eqn-B-08

In [2]:
# extract vector from skew matrix
def anti_skew(x):
    return Matrix([-x[1, 2], x[0, 2], -x[0, 1]])


# do skew matrix from vector


def skew(w):
    return Matrix([[0, -w[2], w[1]], [w[2], 0, -w[0]], [-w[1], w[0], 0]])


In [3]:
t = symbols("t")
mass = 13.7400
# theta = symbols("theta_:3")
theta1 = Function("theta_1")(t)
theta2 = Function("theta_2")(t)
theta3 = Function("theta_3")(t)
theta = Matrix([theta1, theta2, theta3])

p1 = Function("p_1")(t)
p2 = Function("p_2")(t)
p3 = Function("p_3")(t)
p = Matrix([p1, p2, p3])

g = Matrix([0, 0, 9.81])

omega1 = Function("omega_1")(t)
omega2 = Function("omega_2")(t)
omega3 = Function("omega_3")(t)
omega = Matrix([omega1, omega2, omega3])

# I11, I22, I33 = symbols("I_11 I_22 I_33")
# I = diag(*[I11, I22, I33])

m = diag(mass, mass, mass)

# I = Matrix(
#     [
#         [0.0329509600000000, -3.66000000000000e-05, -6.11000000000000e-05],
#         [-3.66000000000000e-05, 0.289971035000000, -2.75000000000000e-05],
#         [-6.11000000000000e-05, -2.75000000000000e-05, 0.314922995000000],
#     ]
# )

I = Matrix([
    [0.0329509600000000, 0, 0],
    [0, 0.289971035000000, 0],
    [0, 0, 0.314922995000000],
])

R1 = Matrix([[cos(theta[2]), -sin(theta[2]), 0],
             [sin(theta[2]), cos(theta[2]), 0], [0, 0, 1]])
R2 = Matrix([[cos(theta[1]), 0, sin(theta[1])], [0, 1, 0],
             [-sin(theta[1]), 0, cos(theta[1])]])
R3 = Matrix([[1, 0, 0], [0, cos(theta[0]), -sin(theta[0])],
             [0, sin(theta[0]), cos(theta[0])]])
R = simplify(R1 @ R2 @ R3)

# omega = R.T @ diff(theta, t)

u_feet1 = symbols("f_1")
u_feet2 = symbols("f_2")
u_feet3 = symbols("f_3")
u_feet4 = symbols("f_4")
u_feet5 = symbols("f_5")
u_feet6 = symbols("f_6")
u_feet7 = symbols("f_7")
u_feet8 = symbols("f_8")
u_feet9 = symbols("f_9")
u_feet10 = symbols("f_10")
u_feet11 = symbols("f_11")
u_feet12 = symbols("f_12")

feet_location1 = symbols("fp_1")
feet_location2 = symbols("fp_2")
feet_location3 = symbols("fp_3")
feet_location4 = symbols("fp_4")
feet_location5 = symbols("fp_5")
feet_location6 = symbols("fp_6")
feet_location7 = symbols("fp_7")
feet_location8 = symbols("fp_8")
feet_location9 = symbols("fp_9")
feet_location10 = symbols("fp_10")
feet_location11 = symbols("fp_11")
feet_location12 = symbols("fp_12")

fs = Matrix([
    u_feet1,
    u_feet2,
    u_feet3,
    u_feet4,
    u_feet5,
    u_feet6,
    u_feet7,
    u_feet8,
    u_feet9,
    u_feet10,
    u_feet11,
    u_feet12,
])
fs = fs.reshape(4, 3).T

fps = Matrix([
    feet_location1,
    feet_location2,
    feet_location3,
    feet_location4,
    feet_location5,
    feet_location6,
    feet_location7,
    feet_location8,
    feet_location9,
    feet_location10,
    feet_location11,
    feet_location12,
])
fps = fps.reshape(4, 3).T


In [4]:
J = zeros(3, 3)
for i in range(3):
    J[:, i] = anti_skew(R.T @ diff(R, theta[i]))
J = simplify(J)
simplify(J.inv())


Matrix([
[1, sin(theta_1(t))*tan(theta_2(t)), cos(theta_1(t))*tan(theta_2(t))],
[0,                 cos(theta_1(t)),                -sin(theta_1(t))],
[0, sin(theta_1(t))/cos(theta_2(t)), cos(theta_1(t))/cos(theta_2(t))]])

checkin if $\hat\omega_B = R^T \dot{R}$ the same as $ \omega_B = J \dot\theta$

In [5]:
simplify(anti_skew(R.T @ diff(R, t))) - simplify(J @ diff(theta, t))


Matrix([
[0],
[0],
[0]])

the right hand side of the dynamics equation

In [6]:
rhs = zeros(4, 3)


$\dot p = \dot p$


In [7]:
rhs[0, :] = diff(p, t).T


$\dot\theta = J^{-1} \hat\omega_B$

In [8]:
rhs[1, :] = simplify(J.inv() @ omega).T


$\ddot{p} = \sum{f}/m - g$

In [9]:
sum_f = zeros(3, 1)
for i in range(4):
    sum_f += fs[:, i]

rhs[2, :] = simplify((sum_f / mass - g).T)


$\dot{\omega_B} = I_B^{-1}(R^T\sum{r_i \times f_i} - \hat \omega_B I_B\omega_B)$


In [10]:
sum_f_m = zeros(3, 1)
for i in range(4):
    r_i = fps[:, i] - p  # foot positions_i
    sum_f_m += r_i.cross(fs[:, i])

rhs[3, :] = simplify(I.inv() @ (R.T @ sum_f_m - skew(omega) @ I @ omega)).T


In [11]:
p_dot = Matrix(symbols("p_dot_1:4"))
rhs_vec = rhs.reshape(12, 1)
for i in range(3):
    rhs_vec = rhs_vec.subs(diff(p[i], t), p_dot[i])
rhs_vec


Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                p_dot_1],
[                                                                                                                                                   

In [12]:
state_sym = p.col_join(theta).col_join(p_dot).col_join(omega)
parameters_sym = fps.T.reshape(12, 1).col_join(fs.T.reshape(12, 1))

state_real = np.linspace(1, 10, 12)
parameters_real = np.linspace(0.4, 4, 24)

sp_rhs_function = lambdify([state_sym, parameters_sym], rhs_vec)
print("sympy function")
print(sp_rhs_function(state_real, parameters_real))
%timeit sp_rhs_function(state_real, parameters_real)
# with open("y.txt", "w") as outf:
#     pickle.dump(m, outf)

sympy function
[[  5.90909091]
 [  6.72727273]
 [  7.54545455]
 [-17.87348989]
 [ -5.65715127]
 [ 28.9946412 ]
 [  0.86829947]
 [  0.91386621]
 [ -8.85056705]
 [-80.01278184]
 [ 96.99788936]
 [ -2.48111842]]
31.5 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [13]:
x = ca.SX.sym("x", 12)
u = ca.SX.sym("u", 24)
ca_rhs = ca.simplify(sp_rhs_function(ca.vertsplit(x), ca.vertsplit(u)))
ca_rhs_fun = ca.Function("rhs", [x, u], [ca_rhs])

print(ca_rhs_fun(state_real, parameters_real).full())

# %timeit -r 20 ca_rhs_fun(state_real, parameters_real)
with open("casadi_rhs.txt", "wb") as outf:
    pickle.dump(ca_rhs_fun, outf)

[[  5.90909091]
 [  6.72727273]
 [  7.54545455]
 [-17.87348989]
 [ -5.65715127]
 [ 28.9946412 ]
 [  0.86829947]
 [  0.91386621]
 [ -8.85056705]
 [-80.01278184]
 [ 96.99788936]
 [ -2.48111842]]


In [14]:
ca_rhs_fun.generate("quad_casadi_dynamics.c")
C = ca.Importer('quad_casadi_dynamics.c', 'shell')
f = ca.external('rhs', C)
print(f(state_real, parameters_real))
# %timeit -r 20 f(state_real, parameters_real)


[5.90909, 6.72727, 7.54545, -17.8735, -5.65715, 28.9946, 0.868299, 0.913866, -8.85057, -80.0128, 96.9979, -2.48112]


In [15]:
with open("casadi_rhs.txt", "rb") as outf:
    ca_rhs = pickle.load(outf)
# %timeit -r 20 ca_rhs(state_real, parameters_real)
ca_rhs(state_real, parameters_real).full()

array([[  5.90909091],
       [  6.72727273],
       [  7.54545455],
       [-17.87348989],
       [ -5.65715127],
       [ 28.9946412 ],
       [  0.86829947],
       [  0.91386621],
       [ -8.85056705],
       [-80.01278184],
       [ 96.99788936],
       [ -2.48111842]])

In [16]:
ca_rhs_fun()

Function(rhs:(i0[12],i1[24])->(o0[12]) SXFunction)

In [59]:
dae = ca.DaeBuilder('vdp')

x_dae = dae.add_x("x")
u_dae = dae.add_u("u")
# x_dae = []
# u_dae = []
# for i in range(12):
#     x_dae.append(dae.add_x("x{}".format(i)))
# for i in range(24):
#     u_dae.append(dae.add_u("u{}".format(i)))

dae.set_ode("x", ca_rhs_fun(x_dae, u_dae))
ca_rhs_fun(x_dae, u_dae)


MX(rhs(x[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], u[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]){0})

In [60]:
funcs = dae.export_fmu()
print('Generated files: {}'.format(funcs))

Generated files: ['vdp.c', 'vdp.h', 'vdp_wrap.c', 'buildDescription.xml', 'modelDescription.xml']


CasADi - 2023-06-23 18:14:28 WARNING("FMU generation is experimental and incomplete") [.../casadi/core/dae_builder_internal.cpp:684]


In [61]:
import os

os.system(
    'gcc --shared -fPIC -I/home/slava/Desktop/Docker/casadi/external_packages/FMI-Standard-3.0/headers/ vdp.c vdp_wrap.c -o vdp.so'
)
print('Compiled vdp.so')
import zipfile
with zipfile.ZipFile('vdp_generated.fmu', 'w') as fmufile:
    # Add generated files to the archive
    for f in funcs:
        arcname = f if f == 'modelDescription.xml' else 'sources/' + f
        fmufile.write(f, arcname=arcname)
        # os.remove(f)
    # Add compile DLL to the archive (assume Linux 64 but)
    fmufile.write('vdp.so', arcname='binaries/x86-linux/vdp.so')
    # os.remove('vdp.so')
print('Created FMU: vdp_generated.fmu')


Compiled vdp.so
Created FMU: vdp_generated.fmu


In [63]:
import zipfile
with zipfile.ZipFile('vdp_generated.fmu', 'r') as zip_ref:
    zip_ref.extractall('vdp_generated')
